In [1]:
import pandas as pd


In [2]:
def delete_duplicates_and_save(input_file, inputs_duplicates, inputs_na):
    df = pd.read_csv(input_file)
    df_unique = df.drop_duplicates(subset=inputs_duplicates)
    df_unique = df_unique.dropna(subset=inputs_na)
    df_unique.to_csv(input_file, index=False)

In [3]:
# delete_duplicates_and_save("SteamAppsList.csv",["appid", "name"], ["appid", "name"])

In [4]:
# delete_duplicates_and_save("SteamGamesMetadata.csv",["appid", "name"], ["appid", "name"])

In [5]:
# delete_duplicates_and_save("SteamPlayersCountHistory.csv", ["appid", "name"], ["appid", "name", "date_playerscount"])

In [6]:
df = pd.read_csv("SteamPlayersCountHistory.csv")
df.info()
df = df.drop_duplicates(subset=["name", "appid"])
df.info()
df = df.dropna(subset=["date_playerscount","name"])
df.info()
df.count()
# df.to_csv("SteamPlayersCountHistory.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10996 entries, 0 to 10995
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   appid              10996 non-null  int64 
 1   name               10996 non-null  object
 2   date_playerscount  10996 non-null  object
dtypes: int64(1), object(2)
memory usage: 257.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10996 entries, 0 to 10995
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   appid              10996 non-null  int64 
 1   name               10996 non-null  object
 2   date_playerscount  10996 non-null  object
dtypes: int64(1), object(2)
memory usage: 257.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10996 entries, 0 to 10995
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   app

appid                10996
name                 10996
date_playerscount    10996
dtype: int64

In [7]:
import time
df = pd.read_csv("SteamPlayersCountHistory.csv")
games_that_dont_have_players = []
currnet_time = int(time.time() * 1000)
one_day_ago = currnet_time - 86400000
one_week_ago = currnet_time - 604800000

for data in df.iterrows():
    split_to_last = data[1]["date_playerscount"].split(", ")[-1]
    current_data_split = split_to_last.split(" ")
    timestamp_ms = int(current_data_split[0])
    if not (one_week_ago <= timestamp_ms <= currnet_time):
        games_that_dont_have_players.append(data[1]["appid"])
        continue

    all_players = []
    for current_data in data[1]["date_playerscount"].split(", "):
        current_data_split = current_data.split(" ")
        players_count = int(current_data_split[1])
        all_players.append(players_count)

    if max(all_players) <= 3:
        games_that_dont_have_players.append(data[1]["appid"])
    if  int(split_to_last.split(" ")[1]) == 0:
        games_that_dont_have_players.append(data[1]["appid"])


In [8]:
df = df[~df["appid"].isin(games_that_dont_have_players)]
df.info()
# df.to_csv("Clean_SteamHistory_PCout.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 9562 entries, 0 to 10995
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   appid              9562 non-null   int64 
 1   name               9562 non-null   object
 2   date_playerscount  9562 non-null   object
dtypes: int64(1), object(2)
memory usage: 298.8+ KB


In [16]:
df = pd.read_csv("SteamGamesMetadata.csv")
df2 = pd.read_csv("Clean_SteamHistory_PCout.csv")
df = df[df["appid"].isin(df2["appid"])]
df.info()
# df.to_csv("SteamGamesMetadata.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9316 entries, 0 to 9315
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   appid              9316 non-null   int64 
 1   name               9316 non-null   object
 2   header_image       9316 non-null   object
 3   short_description  9194 non-null   object
 4   developers         9262 non-null   object
 5   publishers         9230 non-null   object
 6   release_date       9288 non-null   object
 7   platforms          9316 non-null   object
 8   price              7142 non-null   object
 9   categories         9158 non-null   object
 10  genres             9068 non-null   object
 11  website            6303 non-null   object
 12  screenshots        9135 non-null   object
 13  background         9316 non-null   object
dtypes: int64(1), object(13)
memory usage: 1019.1+ KB


In [17]:
bad_apis_txt = "bad_fetching_appids.txt"
bad_apis = []
if bad_apis_txt:
    with open(bad_apis_txt, "r") as f:
        bad_apis = [line.strip() for line in f.readlines()]

df = pd.read_csv("SteamGamesMetadata.csv")
df2 = pd.read_csv("Clean_SteamHistory_PCout.csv")

df2 = df2[df2["appid"].isin(df["appid"])]
df2 = df2[~df2["appid"].isin(bad_apis)]

df2.info()
df2.to_csv("Clean_SteamHistory_PCout.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 9316 entries, 0 to 9561
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   appid              9316 non-null   int64 
 1   name               9316 non-null   object
 2   date_playerscount  9316 non-null   object
dtypes: int64(1), object(2)
memory usage: 291.1+ KB
